In [1]:
!pip install torch torchvision
!pip install adabelief-pytorch

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /global/common/software/nersc9/pytorch/2.6.0/lib/python3.12/site-packages/torchvision-0.21.0+7af6987-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /global/common/software/nersc9/pytorch/2.6.0/lib/python3.12/site-packages/setuptools-75.8.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /global/common/software/nersc9/pytorch/2.6.0/lib/python3.12/site-packages/pillow-11.1.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package install

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from adabelief_pytorch import AdaBelief
import torchvision.models as models
import torch.optim as optim
import random
import scipy.ndimage

In [3]:
!nvidia-smi

Sun Dec 21 08:21:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:03:00.0 Off |                    0 |
| N/A   32C    P0             70W /  400W |    1579MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
import numpy as np
import os

path = "/global/cfs/projectdirs/deepsrch/jwst_sims/pristine_bright/"

# Load data
x0 = np.load(os.path.join(path, "images.npy"))        # (N,H,W)
y0 = np.load(os.path.join(path, "lensed.npy"))        # (N,)
theta_e = np.load(os.path.join(path, "theta_Es.npy")) # (N,)

# Types
x0 = x0.astype(np.float32)
y0 = y0.astype(np.int64)
theta_e = theta_e.astype(np.float32)

# Normalize images (global)
mean = x0.mean()
std  = x0.std() + 1e-8
x0 = (x0 - mean) / std

# === Hard split ===
non_lensed_mask = (y0 == 0)
lensed_mask     = (y0 == 1)

xtrain = x0[non_lensed_mask]
ytrain = y0[non_lensed_mask]
ttrain = theta_e[non_lensed_mask]   # <-- add this

xval = x0
yval = y0
tval = theta_e                      # <-- add this

# Reshape to channels-last for your VQ-VAE code
xtrain = xtrain[..., None]          # (N,125,125,1)
xval   = xval[..., None]

# Keep labels as (N,1) if your code expects that
ytrain = ytrain.reshape(-1, 1)
yval   = yval.reshape(-1, 1)

# Theta: you can keep (N,) or reshape to (N,1) — choose one and stay consistent
ttrain = ttrain.reshape(-1, 1)
tval   = tval.reshape(-1, 1)

print("xtrain:", xtrain.shape, "ytrain:", ytrain.shape, "ttrain:", ttrain.shape)
print("xval:  ", xval.shape,   "yval:  ", yval.shape,   "tval:  ", tval.shape)

xtrain: (10000, 125, 125, 1) ytrain: (10000, 1) ttrain: (10000, 1)
xval:   (20000, 125, 125, 1) yval:   (20000, 1) tval:   (20000, 1)


In [5]:
class Encoder(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=64, z_channels=128):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=4, stride=2, padding=1)  # 125 → 63
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, stride=1, padding=1)  # stays 63
        self.conv3 = nn.Conv2d(hidden_channels, z_channels, kernel_size=3, stride=1, padding=1)  # stays 63

        self.relu = nn.ReLU()

    def forward(self, x):
        out1 = self.relu(self.conv1(x))
        out2 = self.relu(self.conv2(out1))
        z = self.conv3(out2)

        return z, out1 

class Decoder(nn.Module):
    def __init__(self, z_channels=128, hidden_channels=64, out_channels=1):
        super().__init__()

        self.decode_main = nn.Sequential(
            nn.ConvTranspose2d(z_channels, hidden_channels, kernel_size=4, stride=2, padding=1),  # 63 → 126
            nn.ReLU(),
        )

        # After upsampling, combine with skip connection from encoder
        self.final_layers = nn.Sequential(
            nn.Conv2d(hidden_channels + hidden_channels, hidden_channels, kernel_size=3, stride=1, padding=1),  # Skip concat
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=3, stride=1, padding=1)  # Output: (1, 125, 125)
        )

    def forward(self, z, skip_connection):
        x = self.decode_main(z)  # Upsample

        # Crop skip connection to match (in case of size mismatch)
        skip_upsampled = F.interpolate(skip_connection, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, skip_upsampled], dim=1)
        x = self.final_layers(x)

        return x[:, :, :125, :125] 

class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings=64, embedding_dim=32, commitment_cost=0.50):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/self.num_embeddings, 1/self.num_embeddings)
        self.commitment_cost = commitment_cost

    def forward(self, z):
        # z: [B, C, H, W] → [B, H, W, C]
        z_perm = z.permute(0, 2, 3, 1).contiguous()
        z_flattened = z_perm.view(-1, self.embedding_dim)
    
        # Compute distances
        distances = (
            torch.sum(z_flattened**2, dim=1, keepdim=True)
            - 2 * torch.matmul(z_flattened, self.embedding.weight.t())
            + torch.sum(self.embedding.weight**2, dim=1)
        )
    
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.size(0), self.num_embeddings, device=z.device)
        encodings.scatter_(1, encoding_indices, 1)
    
        quantized = torch.matmul(encodings, self.embedding.weight)
        quantized = quantized.view(z_perm.shape)
    
        # Undo permute → back to [B, C, H, W]
        quantized = quantized.permute(0, 3, 1, 2).contiguous()
    
        # Losses (quantized and z now match: [B, C, H, W])
        commitment_loss = self.commitment_cost * F.mse_loss(quantized.detach(), z)
        codebook_loss = F.mse_loss(quantized, z.detach())
    
        # Straight-through estimator
        quantized = z + (quantized - z).detach()
    
        return quantized, commitment_loss + codebook_loss

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 4, stride=2, padding=1),  # (1, 125, 125) → (64, ~63, ~63)
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 1, 4, stride=1, padding=1),  # → (1, ~61, ~61)
        )

    def forward(self, x):
        return self.model(x)

class VQVAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.quantizer = VectorQuantizer()

    def forward(self, noisy_input, clean_target):
        z, skip = self.encoder(noisy_input)
        quantized, vq_loss = self.quantizer(z)

        residual = self.decoder(quantized, skip)
        # Ensure residual matches input size exactly
        residual_upsampled = F.interpolate(residual, size=noisy_input.shape[2:], mode='bilinear', align_corners=False)
        
        x_recon = noisy_input + residual_upsampled
        recon_loss = F.l1_loss(x_recon, clean_target)

        return x_recon, recon_loss + vq_loss

In [6]:
class GalaxyDataset(Dataset):
    def __init__(self, images, noise_std=0.1, add_poisson=True, exp_time_range=(1e2, 1e6), augment=True):
        self.images = images.astype(np.float32).transpose(0, 3, 1, 2)  # (N, 1, 125, 125)
        self.noise_std = noise_std
        self.add_poisson = add_poisson
        self.exp_time_range = exp_time_range  # (min_exp, max_exp)
        self.augment = augment

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]  # Clean image

        # ----- Augmentation Block -----
        if self.augment:
            # Random rotation (0, 90, 180, 270)
            k = random.choice([0, 1, 2, 3])
            img = np.rot90(img, k, axes=(1, 2)).copy()

            # Random flip
            if random.random() < 0.5:
                img = np.flip(img, axis=1).copy()  # Horizontal
            if random.random() < 0.5:
                img = np.flip(img, axis=2).copy()  # Vertical

            # Mild elliptical stretching (affine shear or zoom)
            zoom_factor = np.random.uniform(0.6, 1.4)
            img = scipy.ndimage.zoom(img, (1, zoom_factor, 1), order=1)  # Stretch vertically
            if img.shape[1] > 125:
                img = img[:, :125, :]
            elif img.shape[1] < 125:
                pad = 125 - img.shape[1]
                img = np.pad(img, ((0,0),(0,pad),(0,0)), mode='reflect')

            # Flux scaling
            flux_scale = np.random.uniform(0.8, 1.2)
            img = img * flux_scale

        # ----- Add Noise -----
        noisy_img = img.copy()

        if self.add_poisson:
            exp_time = 10 ** np.random.uniform(np.log10(self.exp_time_range[0]), np.log10(self.exp_time_range[1]))
            sigma = np.sqrt(np.abs(img) / exp_time).astype(np.float32)
            poisson_noise = np.random.normal(0, 1, img.shape).astype(np.float32) * sigma
            noisy_img += poisson_noise

        gaussian_noise = np.random.normal(0, self.noise_std, img.shape).astype(np.float32)
        noisy_img += gaussian_noise

        return torch.tensor(noisy_img, dtype=torch.float32), torch.tensor(img, dtype=torch.float32)


class InferenceDataset(Dataset):
    def __init__(self, images, labels, theta_e=None, arc=None):
        self.images = images.astype(np.float32).transpose(0, 3, 1, 2)
        self.labels = labels
        self.theta_e = theta_e.astype(np.float32) if theta_e is not None else None
        self.arc = arc.astype(np.float32).transpose(0, 3, 1, 2) if arc is not None else None

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx]).long()

        out = [image, label]

        if self.theta_e is not None:
            out.append(torch.tensor(self.theta_e[idx], dtype=torch.float32))  # scalar

        if self.arc is not None:
            arc = torch.tensor(self.arc[idx], dtype=torch.float32)
            out.append(arc)

        return tuple(out)

In [7]:
#hyperparams

epochs = 200
lr= 2.5e-4
patience = 3
model_path = "models/3_channel_vq_vae_2.pth"
bs = 8


In [8]:
import os
import numpy as np
from torch.utils.data import DataLoader

path = "/global/cfs/projectdirs/deepsrch/jwst_sims/pristine_bright/"

# load theta_e for the full dataset (same indexing as xval/yval)
theta_e = np.load(os.path.join(path, "theta_Es.npy")).astype(np.float32)

# Apply nothing to either set
train_dataset = GalaxyDataset(xtrain)
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

# IMPORTANT: InferenceDataset must accept theta_e (we'll update the class accordingly)
inference_dataset = InferenceDataset(xval, yval, theta_e=theta_e)
inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False, num_workers=0)


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize models
model = VQVAE().to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = AdaBelief(model.parameters(), lr=lr, eps=1e-8, betas=(0.9, 0.999),
                        weight_decay=1e-5, rectify=False)
optimizer_D = AdaBelief(discriminator.parameters(), lr=lr, eps=1e-8, betas=(0.9, 0.999),
                        weight_decay=1e-5, rectify=False)

# Loss and GAN weight
bce = nn.BCEWithLogitsLoss()
lambda_adv = 0.0025

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Please check your arguments if you have upgraded adabelief-pytorch from versio

In [11]:
best_val_loss = float('inf')
patience_counter = 0
train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()
    discriminator.train()
    train_loss = 0

    for noisy_batch, clean_batch in tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training"):
        noisy_batch = noisy_batch.to(device)
        clean_batch = clean_batch.to(device)

        # === Generator forward pass ===
        x_recon, vqvae_loss = model(noisy_batch, clean_batch)

        # === Discriminator update ===
        optimizer_D.zero_grad()

        real_preds = discriminator(clean_batch)
        fake_preds = discriminator(x_recon.detach())  # Don't backprop through G

        real_loss = bce(real_preds, torch.ones_like(real_preds))
        fake_loss = bce(fake_preds, torch.zeros_like(fake_preds))
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # === Generator update ===
        optimizer_G.zero_grad()

        # Forward pass again for grad tracking
        x_recon, vqvae_loss = model(noisy_batch, clean_batch)
        fake_preds = discriminator(x_recon)
        adv_loss = bce(fake_preds, torch.ones_like(fake_preds))

        g_loss = vqvae_loss + lambda_adv * adv_loss
        g_loss.backward()
        optimizer_G.step()

        train_loss += g_loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # === Validation ===
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for noisy_batch in inference_loader:
            noisy_batch = noisy_batch[0].to(device)

            x_recon, loss = model(noisy_batch, noisy_batch)
            val_loss += loss.item()

    val_loss /= len(inference_loader)
    val_losses.append(val_loss)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    # === Early stopping based on val loss ===
    if val_loss < best_val_loss - 5e-5:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), model_path)
        print(f"Saved model (val loss: {val_loss:.6f})")
    else:
        patience_counter += 1
        print(f"No val improvement. Patience {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("Early stopping triggered!")
        break

[Epoch 1] Training: 100%|██████████| 1250/1250 [00:29<00:00, 41.85it/s]


[Epoch 1] Train Loss: 0.087708 | Val Loss: 0.005167
Saved model (val loss: 0.005167)


[Epoch 2] Training: 100%|██████████| 1250/1250 [00:29<00:00, 42.04it/s]


[Epoch 2] Train Loss: 0.088857 | Val Loss: 0.005915
No val improvement. Patience 1/3


[Epoch 3] Training: 100%|██████████| 1250/1250 [00:29<00:00, 42.12it/s]


...

[Epoch 4] Train Loss: 0.090250 | Val Loss: 0.006399
No val improvement. Patience 1/3


[Epoch 5] Training: 100%|██████████| 1250/1250 [00:29<00:00, 42.08it/s]


[Epoch 5] Train Loss: 0.090261 | Val Loss: 0.006406
No val improvement. Patience 2/3


[Epoch 6] Training: 100%|██████████| 1250/1250 [00:29<00:00, 42.14it/s]


[Epoch 6] Train Loss: 0.092404 | Val Loss: 0.008639
No val improvement. Patience 3/3
Early stopping triggered!


In [ ]:
# x0_no_lens = np.load(path + "no_lens.npy")

# # Set up inference loader for all 20k
# model.load_state_dict(torch.load(model_path))
# model.eval()
# os.makedirs("combo_triplets_3", exist_ok=True)

# with torch.no_grad():
#     for i, (img, label) in enumerate(tqdm(inference_loader, desc="Generating Triplets")):
#         img = img.to(device)
#         recon, _ = model(img)
#         heatmap = torch.abs(img - recon) * 3
#         triplet = torch.cat([img, recon, heatmap], dim=1).squeeze().cpu().numpy()  # (3, 125, 125)
#         lens_vis = x0_no_lens[i]

#         save_dict = {
#             "triplet": triplet,
#             "label": label.item(),
#             "lens_vis": lens_vis
#         }

#         np.savez_compressed(f"combo_triplets_3/triplet_{i:05d}.npz", **save_dict)

In [12]:
import os
import numpy as np
import torch
from tqdm import tqdm

# Set up inference loader for all 20k
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

out_dir = "combo_triplets_3_U_Net"
os.makedirs(out_dir, exist_ok=True)

with torch.no_grad():
    for i, batch in enumerate(tqdm(inference_loader, desc="Generating Triplets")):
        # batch is (img, label, theta_e) OR (img, label, theta_e, arc)
        img   = batch[0].to(device)          # (1,1,H,W)
        label = int(batch[1].item())         # 0/1
        theta = float(batch[2].item())       # scalar

        recon, _ = model(img, img)           # (1,1,H,W)
        heatmap = torch.abs(img - recon) * 3.0

        triplet = torch.cat([img, recon, heatmap], dim=1) \
                      .squeeze(0).cpu().numpy().astype(np.float32)  # (3,H,W)

        # OPTIONAL: store the actual input image as lens_vis for easy plotting later
        lens_vis_np = img.squeeze(0).squeeze(0).detach().cpu().numpy().astype(np.float32)  # (H,W)

        np.savez_compressed(
            os.path.join(out_dir, f"triplet_{i:05d}.npz"),
            triplet=triplet,
            label=label,
            theta_e=(theta if label == 1 else np.nan),  # theta only meaningful for lenses
            lens_vis=lens_vis_np,
        )

Generating Triplets: 100%|██████████| 20000/20000 [08:18<00:00, 40.10it/s]


In [ ]:
class ComboTripletDataset(Dataset):
    def __init__(self, folder_path, transform=None, return_lens_vis=True):
        self.file_list = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.endswith(".npz")
        ])
        self.transform = transform
        self.return_lens_vis = return_lens_vis

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        data = np.load(self.file_list[idx])
        combo = data["triplet"]  # shape: (3, 125, 125)
        label = int(data["label"])

        combo_tensor = torch.tensor(combo, dtype=torch.float32)

        if self.transform:
            combo_tensor = self.transform(combo_tensor)

        if self.return_lens_vis:
            lens_vis = torch.tensor(data["lens_vis"], dtype=torch.float32).unsqueeze(0)  
            return combo_tensor, label, lens_vis
        else:
            return combo_tensor, label

class ComboTripletDatasetWithNoise(Dataset):
    def __init__(self, folder_path, transform=None, return_lens_vis=True, add_noise=False, noise_std=0.1):
        self.file_list = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.endswith(".npz")
        ])
        self.transform = transform
        self.return_lens_vis = return_lens_vis
        self.add_noise = add_noise
        self.noise_std = noise_std

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        data = np.load(self.file_list[idx])
        combo = data["triplet"]  
        label = int(data["label"])

        combo_tensor = torch.tensor(combo, dtype=torch.float32)  # shape: (3, H, W)

        if self.transform:
            combo_tensor = self.transform(combo_tensor)

        # Add Gaussian noise to all 3 channels
        if self.add_noise:
            noise = torch.randn_like(combo_tensor) * self.noise_std
            combo_tensor = combo_tensor + noise
            combo_tensor = torch.clamp(combo_tensor, 0.0, 1.0)  # optional: clip to keep values valid

        if self.return_lens_vis:
            lens_vis = torch.tensor(data["lens_vis"], dtype=torch.float32).unsqueeze(0)  # (1, 125, 125)
            return combo_tensor, label, lens_vis
        else:
            return combo_tensor, label

In [ ]:
#hyperparams

lr = 5e-5
bs = 8
patience = 40
epochs = 200

In [ ]:
from torchvision.transforms import Resize

transform = transforms.Compose([
    transforms.Lambda(lambda x: transforms.functional.resize(x, (380, 380))),
])

from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# First, get labels for stratified split
temp_dataset = ComboTripletDataset("combo_triplets_3", transform=transform)
indices = list(range(len(temp_dataset)))
labels = [int(np.load(temp_dataset.file_list[i])["label"]) for i in indices]
train_idx, val_idx = train_test_split(indices, test_size=0.2, stratify=labels)

# Create dataset with noise for training
train_dataset = ComboTripletDataset(
    "combo_triplets_3",
    transform=transform,
    return_lens_vis=True,
)
# train_dataset.add_noise = True     # Inject noise flag
# train_dataset.noise_std = 0.1      # Set noise std

# Create clean dataset for validation
val_dataset = ComboTripletDataset(
    "combo_triplets_3",
    transform=transform,
    return_lens_vis=True,
)
# val_dataset.add_noise = False      # No noise on val

# Wrap in Subset based on indices
train_loader = DataLoader(Subset(train_dataset, train_idx), batch_size=16, shuffle=True)
val_loader = DataLoader(Subset(val_dataset, val_idx), batch_size=16, shuffle=False)

In [ ]:
from collections import Counter

# === Check dataset size split ===
total = len(train_dataset)
print(f"Total samples: {total}")
print(f"Train size: {len(train_idx)} ({len(train_idx)/total:.2%})")
print(f"Val size:   {len(val_idx)} ({len(val_idx)/total:.2%})")

# === Check label balance ===
train_labels = [int(np.load(train_dataset.file_list[i])["label"]) for i in train_idx]
val_labels = [int(np.load(val_dataset.file_list[i])["label"]) for i in val_idx]

print("\nTrain Label Distribution:", Counter(train_labels))
print("Val Label Distribution:  ", Counter(val_labels))

# === Check shape of a batch ===
images, labels, _ = next(iter(train_loader))
print("\nSample batch shape:", images.shape)  # Expect (8, 3, 380, 380)
print("Sample labels:", labels)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. Load EfficientNetV2-L
efficientnet = models.efficientnet_v2_l(weights="EfficientNet_V2_L_Weights.DEFAULT")

# 3. Modify final classifier for binary classification
num_features = efficientnet.classifier[1].in_features
efficientnet.classifier[1] = nn.Linear(num_features, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 4. Move model to device
efficientnet = efficientnet.to(device)

# 5. Loss + Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdaBelief(efficientnet.parameters(), lr=lr, eps=1e-8, betas=(0.9, 0.999), weight_decay=1e-5, rectify=True)

In [ ]:
from sklearn.metrics import recall_score

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
val_recalls = []

best_val_recall = 0.0
best_val_acc = 0.0
patience_counter = 0

best_model_path = "models/effnetv2l_best_2.pth"

for epoch in range(epochs):
    # === Training ===
    efficientnet.train()
    train_loss = 0
    correct = 0

    for images, labels, _ in tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = efficientnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()

    train_acc = correct / len(train_loader.dataset)
    train_loss /= len(train_loader)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # === Validation ===
    efficientnet.eval()
    val_loss = 0
    correct = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = efficientnet(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = correct / len(val_loader.dataset)
    val_loss /= len(val_loader)
    recall = recall_score(all_labels, all_preds, pos_label=1)

    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    val_recalls.append(recall)

    print(f"[Epoch {epoch+1}] "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, Recall (Lensed): {recall:.4f}")
    
    # === Early stopping
    save_model = False
    
    if recall > best_val_recall + 1e-4:
        best_val_recall = recall
        save_model = True
        print(f"New best recall: {recall:.4f}")
    
    elif recall >= 1.0 and val_acc > best_val_acc:
        best_val_acc = val_acc
        save_model = True
        print(f"Perfect recall maintained; new best acc: {val_acc:.4f}")
    
    if save_model:
        patience_counter = 0
        torch.save(efficientnet.state_dict(), best_model_path)
        print("Model saved.")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}")
    
    if patience_counter >= patience:
        print("Early stopping triggered!")
        break

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Load best model
efficientnet.load_state_dict(torch.load("models/effnetv2l_best_2.pth"))
efficientnet.eval()

all_probs = []
all_preds = []
all_labels = []

# === Inference ===
with torch.no_grad():
    for images, labels, _ in tqdm(val_loader):
        images = images.to(device)
        outputs = efficientnet(images)

        probs = torch.softmax(outputs, dim=1)[:, 1]  # class 1 = lensed
        preds = outputs.argmax(dim=1)

        all_probs.extend(probs.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# === ROC AUC ===
roc_auc = roc_auc_score(all_labels, all_probs)
print(f"ROC AUC: {roc_auc:.4f}")

# === Confusion Matrix ===
cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:")
print(cm)

# === Classification Report ===
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=4))

# === Plot Confusion Matrix ===
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Non-Lensed", "Lensed"],
            yticklabels=["Non-Lensed", "Lensed"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Threshold = 0.5)")
plt.show()

# === Plot ROC Curve ===
fpr, tpr, _ = roc_curve(all_labels, all_probs)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.8f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Load file paths
triplet_dir = "combo_triplets_3"
triplet_files = sorted([
    os.path.join(triplet_dir, f) for f in os.listdir(triplet_dir) if f.endswith(".npz")
])

# Collect 10 of each class
lensed_files = []
non_lensed_files = []

for f in triplet_files:
    data = np.load(f)
    label = int(data["label"])
    if label == 1 and len(lensed_files) < 10:
        lensed_files.append(f)
    elif label == 0 and len(non_lensed_files) < 10:
        non_lensed_files.append(f)
    if len(lensed_files) == 10 and len(non_lensed_files) == 10:
        break

# === OG visual triplet plot ===
def plot_triplet(filepath, title_prefix=""):
    data = np.load(filepath)
    triplet = data["triplet"]
    original, recon, heatmap = triplet
    lens_vis = data["lens_vis"] if "lens_vis" in data else None

    fig, axs = plt.subplots(1, 4, figsize=(12, 3))

    im0 = axs[0].imshow(original, cmap="gray")
    axs[0].set_title(f"{title_prefix}Original")
    axs[0].axis("off")
    fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)

    im1 = axs[1].imshow(recon, cmap="gray")
    axs[1].set_title("Reconstruction")
    axs[1].axis("off")
    fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)

    im2 = axs[3].imshow(heatmap, cmap="hot", vmin=0, vmax=30000)
    axs[3].set_title("Heatmap")
    axs[3].axis("off")
    fig.colorbar(im2, ax=axs[3], fraction=0.046, pad=0.04)

    if lens_vis is not None:
        im3 = axs[2].imshow(lens_vis, cmap="gray")
        axs[2].set_title("No-Lens Ref")
    else:
        im3 = axs[2].imshow(np.zeros_like(original), cmap="gray")
        axs[2].set_title("No Ref")
    axs[2].axis("off")
    fig.colorbar(im3, ax=axs[2], fraction=0.046, pad=0.04)

    plt.tight_layout()

# === Display Lensed ===
print("Showing 10 Lensed Examples")
for file in lensed_files:
    plot_triplet(file, title_prefix="Lensed: ")
    plt.show()

# === Display Non-Lensed ===
print("Showing 10 Non-Lensed Examples")
for file in non_lensed_files:
    plot_triplet(file, title_prefix="Non-Lensed: ")
    plt.show()

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# === Step 1: Load file paths ===
triplet_dir = "combo_triplets_3"
triplet_files = sorted([
    os.path.join(triplet_dir, f) for f in os.listdir(triplet_dir) if f.endswith(".npz")
])

# Collect 10 of each class
lensed_files = []
non_lensed_files = []

for f in triplet_files:
    data = np.load(f)
    label = int(data["label"])
    if label == 1 and len(lensed_files) < 10:
        lensed_files.append(f)
    elif label == 0 and len(non_lensed_files) < 10:
        non_lensed_files.append(f)
    if len(lensed_files) == 10 and len(non_lensed_files) == 10:
        break

# === Step 2: Define binning function ===
def get_pixel_bins(recon, normalize=True):
    pixels = recon.ravel()
    total = pixels.size
    bins = {}

    # Create 20 bins: 0.00–0.05, 0.05–0.10, ..., 0.95–1.00
    edges = np.arange(0.0, 1.01, 0.05)  # include 1.00

    for i in range(len(edges) - 1):
        lower = edges[i]
        upper = edges[i + 1]
        key = f"{lower:.2f}–{upper:.2f}"
        # Include right edge only on the final bin
        count = np.sum((pixels >= lower) & (pixels < upper)) if upper < 1.0 else np.sum((pixels >= lower) & (pixels <= upper))
        bins[key] = count / total if normalize else count

    return bins

# === Step 3: Apply to each file ===
def collect_stats(file_list):
    stat_list = []
    for f in file_list:
        recon = np.load(f)["triplet"][1]
        stat_list.append(get_pixel_bins(recon))
    return pd.DataFrame(stat_list)

lensed_df = collect_stats(lensed_files)
non_lensed_df = collect_stats(non_lensed_files)

# === Step 4: Compute average bin percentages ===
mean_lensed = lensed_df.mean()
mean_non_lensed = non_lensed_df.mean()

# === Step 5: Plot comparison ===
x = np.arange(len(mean_lensed))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, mean_lensed, width, label='Lensed')
plt.bar(x + width/2, mean_non_lensed, width, label='Non-Lensed')
plt.xticks(x, mean_lensed.index, rotation=45)
plt.ylabel("Fraction of pixels per image (normalized)")
plt.title("Pixel Value Distribution Comparison")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# === Reuse your binning function ===
def get_pixel_bins(recon, normalize=True):
    pixels = recon.ravel()
    total = pixels.size
    bins = {}
    edges = np.arange(0.0, 1.01, 0.05)

    for i in range(len(edges) - 1):
        lower = edges[i]
        upper = edges[i + 1]
        key = f"{lower:.2f}–{upper:.2f}"
        count = np.sum((pixels >= lower) & (pixels < upper)) if upper < 1.0 else np.sum((pixels >= lower) & (pixels <= upper))
        bins[key] = count / total if normalize else count

    return bins

# === Individual plots for lensed ===
for i, f in enumerate(lensed_files):
    recon = np.load(f)["triplet"][1]
    bins = get_pixel_bins(recon)
    plt.figure(figsize=(10, 4))
    plt.bar(bins.keys(), bins.values(), color='blue')
    plt.title(f"Lensed {i+1} — Pixel Value Distribution")
    plt.xlabel("Pixel Value Bin")
    plt.ylabel("Fraction of Pixels")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# === Individual plots for non-lensed ===
for i, f in enumerate(non_lensed_files):
    recon = np.load(f)["triplet"][1]
    bins = get_pixel_bins(recon)
    plt.figure(figsize=(10, 4))
    plt.bar(bins.keys(), bins.values(), color='orange')
    plt.title(f"Non-Lensed {i+1} — Pixel Value Distribution")
    plt.xlabel("Pixel Value Bin")
    plt.ylabel("Fraction of Pixels")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()